In [62]:
# SHIFT + ENTER executa e cria nova linha de execução
# ferramentas:
# www.kaggle.com
# https://colab.research.google.com
#  PYTHON, Pandas, Numpy
# https://www.kaggle.com/tedllh/titanic-train
import pandas as pd
import numpy as np
# algorítmo de classificação para construção do modelo
from sklearn.ensemble import RandomForestClassifier
# função auxiliar que ajuda a separar o banco de dados em treino e validação
from sklearn.model_selection import train_test_split

In [63]:
dados = pd.read_csv('/content/train.csv')

In [64]:
dados.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [65]:
# retira informações desnecessárias para o modelo
dados = dados.drop(['Name', 'Ticket', 'Cabin', 'Embarked'], axis=1)

In [66]:
dados.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,1,0,3,male,22.0,1,0,7.2500
1,2,1,1,female,38.0,1,0,71.2833
2,3,1,3,female,26.0,0,0,7.9250
3,4,1,1,female,35.0,1,0,53.1000
4,5,0,3,male,35.0,0,0,8.0500


In [67]:
# editando chave e variável resposta
dados = dados.set_index(['PassengerId'])
dados = dados.rename(columns={'Survived': 'target'}, inplace=False)

In [68]:
# compare com a execução anterior deste mesmo método para entender melhor
# redefine o indice da base de dados
# renomeia o alvo / objetivo / meta da analise
dados.head()

,target,Pclass,Sex,Age,SibSp,Parch,Fare
PassengerId,,,,,,,
1,0,3,male,22.0,1,0,7.2500
2,1,1,female,38.0,1,0,71.2833
3,1,3,female,26.0,0,0,7.9250
4,1,1,female,35.0,1,0,53.1000
5,0,3,male,35.0,0,0,8.0500


In [69]:
# descrição geral da tabela
dados.describe()

,target,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [70]:
# inclui variáveis não numéricas. argumento em MAIÚSCULO
dados.describe(include=['O'])

,Sex
count,891
unique,2
top,male
freq,577


In [71]:
# a maioria dos algorítmos de machine learning não aceitam dados categóricos (strings).
# trocando o que é feminino por 1 e masculino por 0
# criar 3 colunas. uma para cada classe.
# quando Pclass == 1, terá 1 nesta coluna e zero nas outras.
# quando Pclass == 2, terá 1 nesta coluna e zero nas outras.
# quando Pclass == 3, terá 1 nesta coluna e zero nas outras.
dados['Sex_F'] = np.where(dados['Sex'] == 'female', 1, 0)
dados['Pclass_1'] = np.where(dados['Pclass'] == 1, 1, 0)
dados['Pclass_2'] = np.where(dados['Pclass'] == 2, 1, 0)
dados['Pclass_3'] = np.where(dados['Pclass'] == 3, 1, 0)

In [72]:
#  remove as variáveis que foram derivadas.
dados = dados.drop(['Pclass', 'Sex'], axis=1)

In [73]:
dados.head()

,target,Age,SibSp,Parch,Fare,Sex_F,Pclass_1,Pclass_2,Pclass_3
PassengerId,,,,,,,,,
1,0,22.0,1,0,7.2500,0,0,0,1
2,1,38.0,1,0,71.2833,1,1,0,0
3,1,26.0,0,0,7.9250,1,0,0,1
4,1,35.0,1,0,53.1000,1,1,0,0
5,0,35.0,0,0,8.0500,0,0,0,1


In [74]:
# informa se a tabela tem columas com valores que não foram preenchidos (missing).
# no caso age tem 177 valores não preenchidos
dados.isnull().sum()

target        0
Age         177
SibSp         0
Parch         0
Fare          0
Sex_F         0
Pclass_1      0
Pclass_2      0
Pclass_3      0
dtype: int64

In [75]:
# substituie valores
dados.fillna(0, inplace=True)

In [76]:
# substitue valores missing da coluna age por zero. poderia trocar por uma media de idade por exemplo.
dados.isnull().sum()

target      0
Age         0
SibSp       0
Parch       0
Fare        0
Sex_F       0
Pclass_1    0
Pclass_2    0
Pclass_3    0
dtype: int64

In [77]:
# definindo porcentagem de treino e de teste
# algorítimo de amostragem
# segmentar base de dados totais e testes e amostras
# 70% para treino do modelo e 30% para testes do modelo
# porque não treinar o modelo inteiro? RESPOSTA:
# para que o modelo não fique tão específico para a base como um todo, mas que funcione também para novas informações
# retira-se uma uma parte do banco de dados para que o modelo não veja estas informações.
# essa parte retirada será avaliada depois como se estivesse simulando um dado produtivo
# separa a base e como resposta tem uma tabela de treino e outra para teste.
x_train, x_test, y_train, y_test = train_test_split(dados.drop(['target'], axis=1),
                                                    dados["target"],
                                                    test_size = 0.3,
                                                    random_state = 1234)
[{'treino': x_train.shape}, {'teste': x_test.shape}]

[{'treino': (623, 8)}, {'teste': (268, 8)}]

In [78]:
# modelo escolhido foi o Random Forest

# linha abaixo faz o mapeamento do modelo
rndforest = RandomForestClassifier(n_estimators=1000,
                                   criterion='gini',
                                   max_depth=5)

# linha abaixo faz o cálculo do modelo
rndforest.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=5, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [79]:
# calcula as chances de sobrevivencia e depois classifica.
probabilidade = rndforest.predict_proba(dados.drop('target', axis=1))[:,1]
classificacao = rndforest.predict(dados.drop('target', axis=1))

In [80]:
# cria coluna de probabilidade e de classificação
# 
dados['probabilidade'] = probabilidade
dados['classificacao'] = classificacao

In [81]:
# não existe modelo ruim. existe modelo com pouca informação.
# para melhorar este modelo basta adicionar mais dados/atributos/informações.
dados

,target,Age,SibSp,Parch,Fare,Sex_F,Pclass_1,Pclass_2,Pclass_3,probabilidade,classificacao
PassengerId,,,,,,,,,,,
1,0,22.0,1,0,7.2500,0,0,0,1,0.128904,0
2,1,38.0,1,0,71.2833,1,1,0,0,0.933848,1
3,1,26.0,0,0,7.9250,1,0,0,1,0.440749,0
4,1,35.0,1,0,53.1000,1,1,0,0,0.913639,1
5,0,35.0,0,0,8.0500,0,0,0,1,0.137484,0
...,...,...,...,...,...,...,...,...,...,...,...
887,0,27.0,0,0,13.0000,0,0,1,0,0.123599,0
888,1,19.0,0,0,30.0000,1,1,0,0,0.858861,1
889,0,0.0,1,2,23.4500,1,0,0,1,0.485379,0


In [ ]:
# 
